In [1]:
%load_ext autoreload

In [22]:
%matplotlib inline
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

import pandas as pd
import seaborn as sns
import numpy as np

import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split, KFold
from sklearn.grid_search import GridSearchCV

import transform_utils as tu
from kaggle_helper import KaggleHelper

In [23]:
kh = KaggleHelper("submissions.db", "predictions")

table submissions already exists
table validation_scores already exists


In [24]:
%aimport transform_utils
%aimport kaggle_helper

In [25]:
%aimport

Modules to reload:
kaggle_helper transform_utils

Modules to skip:



In [26]:
raw = pd.read_csv("data/train.csv")
print("Unique UPC:", len(raw.Upc.unique()))
print("Unique DepartmentDescription:", len(raw.DepartmentDescription.unique()))
print("Unique FinelineNumber:", len(raw.FinelineNumber.unique()))
raw.head()

Unique UPC: 97715
Unique DepartmentDescription: 69
Unique FinelineNumber: 5196


,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


In [27]:
start = datetime.datetime.now()
%autoreload
df = tu.preprocess_walmart(raw, True)

print("Total Time:", (datetime.datetime.now() - start).total_seconds())

Total Time: 6.813662


In [28]:
df.columns

Index(['DepartmentDescription_PLUS AND MATERNITY',
       'DepartmentDescription_HOME MANAGEMENT',
       'DepartmentDescription_PHARMACY RX', 'DepartmentDescription_AUTOMOTIVE',
       'DepartmentDescription_BEDDING',
       'DepartmentDescription_MEDIA AND GAMING',
       'DepartmentDescription_SHEER HOSIERY',
       'DepartmentDescription_OTHER DEPARTMENTS',
       'DepartmentDescription_LARGE HOUSEHOLD GOODS', 'Weekday_Wednesday',
       'DepartmentDescription_CAMERAS AND SUPPLIES',
       'DepartmentDescription_PERSONAL CARE',
       'DepartmentDescription_MENS WEAR',
       'DepartmentDescription_BOOKS AND MAGAZINES',
       'DepartmentDescription_HORTICULTURE AND ACCESS',
       'DepartmentDescription_GIRLS WEAR, 4-6X  AND 7-14', 'Weekday_Monday',
       'DepartmentDescription_FINANCIAL SERVICES',
       'DepartmentDescription_LADIESWEAR', 'DepartmentDescription_BAKERY',
       'DepartmentDescription_OPTICAL - FRAMES',
       'DepartmentDescription_CONCEPT STORES',
       'Depar

In [29]:
X = df.drop("TripType", axis=1).drop("VisitNumber", axis=1)
sorted_columns = sorted(X.columns)
X = X[sorted_columns]
y = df.TripType

In [30]:
XTrain, XTest, yTrain, yTest = train_test_split(X.values, y.values, test_size=0.2, random_state=200)
XTrain, XVal, yTrain, yVal = train_test_split(XTrain, yTrain, test_size=0.2, random_state=200)

http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html#sklearn.grid_search.GridSearchCV

In [46]:
start = datetime.datetime.now()

clf = RandomForestClassifier(max_features=0.15, n_estimators=50, n_jobs=3)
clf.fit(XTrain, yTrain)

score = clf.score(XVal, yVal)
print(score)

print("Total Time:", (datetime.datetime.now() - start).total_seconds())

0.662202769794
Total Time: 12.900172


In [33]:
kh.save_validation_score(clf, score, len(XTrain))

Enter Notes to save to DB: added in weekdays as well as department etc


# Only Kaggle Exists Below Here

In [34]:
kaggleTest = tu.preprocess_walmart(pd.read_csv("data/test.csv"))
missing_columns = set(sorted_columns).difference(set(kaggleTest.columns))
print("missing columns")
print("-"*15)
for col in missing_columns:
    print(col)
    kaggleTest[col] = 0
kaggleTest = kaggleTest[sorted_columns]
assert list(X.columns) == list(kaggleTest.columns)

missing columns
---------------
DepartmentDescription_HEALTH AND BEAUTY AIDS


In [35]:
%autoreload
test_predictions = clf.predict(kaggleTest.values)
output = tu.convert_predictions(test_predictions, 
                               output_index=pd.Series(kaggleTest.index), 
                               actual_trip_types=df.TripType.unique())

Missing Categories {14}


In [36]:
kh.save_test_predictions(output, clf, len(XTrain))

Enter Notes to save to DB: added in weekdays as well as department etc
